In [ ]:
'''
===== Descriptions
Read in images, do the thinning on the images, extract 0-dim topological
features from the results.
'''
from skimage.morphology import skeletonize, thin
from skimage import data
import matplotlib.pyplot as plt
from skimage.util import invert
import cv2
import copy

%run Viewer.ipynb
%run Topo_treatment.ipynb
%run Archpool.ipynb

img = [None] * 3
img[0] = cv2.imread("D:/Data/cremi_exp/cremi_gen/gen_00098.png", cv2.IMREAD_GRAYSCALE)
img[1] = cv2.imread("D:/Data/cremi_exp/cremi_gen/gen_00099.png", cv2.IMREAD_GRAYSCALE)

img_ = copy.copy(img[1])
img_[img_ < 127.5] = 0
img_[img_ >= 127.5] = 1
skeleton = thin(1 - img_)
img_ = 1 - skeleton
img_[img_ == 1] = 255
img_[img_ == 0] = 0
img[2] = img_

img = Utility_general.normalize_data_(img, 127.5)
img = np.asarray(img)
img_bin = Utility_topo.binarize_data(img, 0.0)

adv_params = return_advanced_params()
et = Edges_(adv_params, False)
B_, D_, PD_, tsfm_list, bnd_ph, red_ph = et.pd_batch(img_bin, 0, debug=True)

Viewer.imshow_(img_bin[2])
x_, y_ = Viewer.draw_bnd_or_red_on_single_dim(img_bin[2], red_ph[2], -1)

In [ ]:
'''
Select two end points on a 64 x 64 images and find the line segment between
them. The thickness can be adjusted.
'''
%run Viewer.ipynb
%run Utility_general.ipynb
import time

aa = np.array([[16, 55], [17, 56]])
bb = Utility_general.connect_2pts(aa, (64, 64), 1)
x_, y_ = Utility_general.connect_2pts(aa, (64, 64), 3, split_form=True)
img = np.ones((64, 64)) * 255
print(bb)

for i in range(bb.shape[0]):
    img[bb[i,1], bb[i,0]] = 0
Viewer.imshow_(img)

In [ ]:
'''
Generate a random distance matrix, solve the linear programming problem
and return the match (mapping)
'''
%run Topo_treatment.ipynb
import numpy as np

dist = np.random.random_sample((3, 3))
lpsolver = LPSolver_(3, 3, False)
sol = lpsolver.Wasserstein_LP(dist)
mapping = lpsolver.approx_OT(sol)
print(mapping)

In [ ]:
'''
Read batch from generated images and a subset of the computed persistence
diagrams. Compute persistence diagrams on the images, filter the pds, and
extract only birth time. Computer 1-wasserstein distance and retrieve the
mappings via linear programming. Find the dot-to-dot mappings and compute
topo force. Compute boundaries and reductions using opencv and finally f-
ind the points to be fixed and plot these points on the original images.
This example is an exact copy of the core topology component in the GAN.
'''

%run Archpool.ipynb
%run Data.ipynb
%run Viewer.ipynb
%run Topo_treatment.ipynb
%run FileIO.ipynb

adv_params = return_advanced_params()
et = Edges_(adv_params, False)

img_bat = Utility_general.read_image_subset("D:/Data/cremi_exp/cremi_gen", "png", 1.0, shuffle=False)
img_bat = Utility_general.normalize_data_(img_bat, 127.5)
img_bat = np.asarray(img_bat)
img_bin = Utility_topo.binarize_data(img_bat, 0.0)

B_, D_, set1, _, bnd_ph, red_ph = et.pd_batch(img_bin, 1, debug=True, old_form=False, binarize=False)
set2, f2        = FileIO.read_pd_subset("D:/Data/cremi_patch64/pds", "dat", 0.005, shuffle=False)
B_, D_, set1    = Utility_topo.topo_filter_retmat_mul(B_, D_, set1, 1.0)
set2            = Utility_topo.topo_filter_retmat(set2, 1.0)

b1 = Utility_topo.extract_dim_from_list(set1, 0)
b2 = Utility_topo.extract_dim_from_list(set2, 0)
lp_ = LPSolver_(len(b1), len(b2), False)
dist, G = Utility_general.wasserstein_set_distance(b1, b2, 1.0)
mapping, dist, G = lp_.linear_program_(dist, G)
force = Utility_topo.topo_force_(b1, b2, G, mapping)

bnd_cv, hcy_cv, red_cv = Utility_topo.compute_bnd_red_cv_batch(img_bin)
flt_list, frc_x_, frc_y_ = Utility_topo.apply_force_(b1, force, B_,
                           bnd_cv, hcy_cv, 2, True, True, True, (64, 64), 3)
#img_frc = et.plot_on_images(img_bat, frc_x_, frc_y_, -1.0)

idx = 20
idxx = 0

print("%d structures in image %d" %(len(frc_x_[idx]), idx))
print('%d points in strcture %d' %(len(frc_x_[idx][idxx]), idxx))

print(frc_x_[idx][idxx])
print(frc_y_[idx][idxx])
print(B_[idx][flt_list[idx][idxx],:])

Viewer.draw_coord_on_image([B_[idx][flt_list[idx][idxx],0]], [B_[idx][flt_list[idx][idxx],1]],
                           img_bin[idx], (255, 0, 0), color=True)
Viewer.draw_coord_on_image(frc_x_[idx][idxx], frc_y_[idx][idxx],
                           img_bin[idx], (0, 0, 0), color=True)
Viewer.draw_bnd_on_single_dim_cv(img_bin[idx], bnd_cv[idx], (255, 0, 0), -1)

'''
sub-example: find line segment between two points and plot
'''
# ends = np.array([B_[idx][flt_list[idx][idxx],:], [frc_x_[idx][idxx][1], frc_y_[idx][idxx][1]]])
# x_, y_ = Utility_general.connect_2pts(ends, (64, 64), 5, split_form=True)
# Viewer.draw_coord_on_image(x_, y_, img_bin[idx], (255, 0, 0), color=True)

In [ ]:
'''
Similiar to example above, except that it calls fix_with_topo routine.
'''

%run Archpool.ipynb
%run Data.ipynb
%run Viewer.ipynb
%run Topo_treatment.ipynb
%run FileIO.ipynb

adv_params = return_advanced_params()
et = Edges_(adv_params, False)

img_bat = Utility_general.read_image_subset("D:/Data/cremi_exp/cremi_gen", "png", 1.0, shuffle=False)
img_bat = Utility_general.normalize_data_(img_bat, 127.5)
img_bat = np.asarray(img_bat)
et.load_pd_pool("D:/Data/cremi_patch64/pds", "dat", 1.0, 101, 0)
gen_res, mean_wasdis = et.fix_with_topo(img_bat, 1, -1.0, 1.0, blind=True)

idx = 0
Viewer.imshow_(img_bat[idx])
Viewer.imshow_(gen_res[idx])

In [ ]:
'''
Example calls of some functions.
'''
# noise generation and write out
fixed_z_ = torch.randn([batch_size]+self.inputG_dims, device=self.device)
FileIO.write_binary('D:/Data/fixed_z/128_128_1_1_0.dat', fixed_z_.cpu().numpy().flatten(), list(fixed_z_.shape), 'f')

# write out images
Dfake_device_ = self.netG(fixed_z_)
FileIO.save_image_batch(Dfake_device_.detach().cpu().numpy(), 'D:/tmp', 'gen', 127.5, 0)

In [ ]:
'''
This block converts the mask ranging from 0 to 255 to binary mask with 0 or 1
for Unet segmentation purpose.
'''
%run FileIO.ipynb

'''
Note Unet requires training images named as cremi_0001.jpg and mask named as cremi_0001_mask.gif
while pix2pix requires same names in two folders. Unet needs to convert masks, pix2pix doesn't.
'''

FileIO.make_mask_unet("D:/Data/cremi/cremi64_mask", "D:/Data/cremi/cremi64_maskunet", 'jpg', 'gif')

'''
===== Launch docker container:
nvidia-docker run -it --name pix2pix -p 8097:8097 -v ~/ptd-shared:/root/shared taesungp/pytorch-cyclegan-and-pix2pix bash
===== Generate training images using files in !shared folder!:
python datasets/combine_A_and_B.py --fold_A ~/shared/dataset/cremi/cremi64_tx --fold_B ~/shared/dataset/cremi/cremi64_maskp2p --fold_AB ~/shared/dataset/cremi/cremi64_combined
python datasets/combine_blank_and_B.py --fold_B ~/shared/dataset/cremi/match0.0005 --fold_AB ~/shared/dataset/cremi/match0.0005_combined
===== Train using files from !workplace!:
python -m visdom.server &
python train.py --dataroot ~/shared/dataset/cremi/cremi64_combined --name cremi64_pix2pix --model pix2pix --netG resnet_9blocks --direction BtoA --resize_or_crop none --gpu_ids '4,5,6,7' --batch_size 8 --input_nc 1 --output_nc 1
===== Test using files from !workplace!:
python test.py --dataroot ~/shared/dataset/cremi/match0.0005_combined/ --name cremi64_pix2pix --model pix2pix --direction BtoA --input_nc 1 --output_nc 1 --resize_or_crop none --netG resnet_9blocks --gpu_ids '2,3' --num_test 6000 --display_winsize 64

===== Segmentation training:
python3 train.py --images /root/shared/dataset/cremi/cremi64_tx/train/ --masks /root/shared/dataset/cremi/cremi64_maskunet/train/ -e 20 -b 32 -l 0.005 -s 1.0 -c 1
python3 predict.py --infolder ../a1 --outfolder ../a2 -m checkpoints/CP_epoch19.pth -c 1 -s 1.0
'''

In [ ]:
'''
Compute and save persistence diagrams for training images
'''
%run FileIO.ipynb
%run Archpool.ipynb

root = 'D:/Data/isbi_patch64'
adv_params = return_advanced_params()
FileIO.compute_pd_save(root, root+'/pds', "png", adv_params, 1)

In [ ]:
%run Viewer.ipynb
%run Archpool.ipynb
# %run Utility_topo.ipynb
# %run Utility_general.ipynb
%run Topo_treatment.ipynb
%run fid/fid_score.ipynb
# %run cyckernel/Cyckernel.ipynb
#import numpy as np
#import cv2

# root0 = 'D:/Data/cremi_patch64/cremi_01103.png'
# root1 = 'D:/Data/cremi_patch64/cremi_01104.png'
# img0 = cv2.imread(root0, cv2.IMREAD_GRAYSCALE)
# img1 = cv2.imread(root1, cv2.IMREAD_GRAYSCALE)
# # img[img > 0.0] = 255.0
# # img[img <= 0.0] = 0.0
# img0 = img0.astype(np.uint8)
# img1 = img1.astype(np.uint8)

# trfm = Utility_general.nz2_nearest_z(img)
# print(np.amax(trfm), np.amin(trfm))

# adv_params = return_advanced_params()
# et = Edges_(adv_params, False)
# dist = et.fd_wasserstein_distance('D:/Data/cremi_patch64', 'D:/Data/cremi_patch64', 'png', 1, 1.0, topothresh=1.0)
# #dist = et.fd_cycle_distance('D:/Data/cremi_patch64', 'D:/Data/cremi_new/compare0.0005', 'png', topothresh=1.0)
# act = np.mean(dist, 1)
# mu = np.mean(act, axis=0)
# sigma = np.cov(act, rowvar=False)
# print(mu, sigma)

# act = np.max(dist, 1)
# mu = np.mean(act, axis=0)
# sigma = np.cov(act, rowvar=False)
# print(mu, sigma)

# act = np.min(dist, 1)
# mu = np.mean(act, axis=0)
# sigma = np.cov(act, rowvar=False)
# print(mu, sigma)

fid = calculate_frechet_distance(1e-5, 1e-5, 58.36175341348612, 1289.0174342325877)
print(fid)